# SETUP MÔI TRƯỜNG

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import seaborn as sns
from pyvi import ViTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LOAD DATASET

In [3]:
df = pd.read_csv('Dataset.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53451 entries, 0 to 53450
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             53451 non-null  object
 1   publishedAtDate   53451 non-null  object
 2   originalLanguage  26707 non-null  object
 3   textTranslated    5386 non-null   object
 4   text              26627 non-null  object
 5   stars             53451 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


In [5]:
df.shape

(53451, 6)

In [5]:
df.head()

,title,publishedAtDate,originalLanguage,textTranslated,text,stars
0,Siêu thị Co.opmart Nha Trang,2025-06-14T08:20:44.449Z,vi,NaN,"Khu vui chơi nóng mặc dù rất ít trẻ nhỏ chơi, ...",1
1,Siêu thị Co.opmart Nha Trang,2025-09-05T01:15:13.880Z,vi,NaN,cái quầy giữ đồ của khách làm ăn khó chịu thật...,1
2,Siêu thị Co.opmart Nha Trang,2025-10-18T02:34:51.862Z,vi,NaN,Mua hàng online. Mặt hàng để còn hàng nhưng lú...,1
3,Siêu thị Co.opmart Nha Trang,2025-06-21T12:36:28.800Z,vi,NaN,"Bán hàng thì đắt hon các siêu thị khác, giữ xe...",1
4,Siêu thị Co.opmart Nha Trang,2025-07-07T12:21:30.734Z,vi,NaN,- Siêu thị duy nhất thu tiền giữ xe\n- Nhân vi...,1


In [6]:
df.tail()

,title,publishedAtDate,originalLanguage,textTranslated,text,stars
53446,Co.opmart Thủ Dầu Một,2025-05-21T13:55:15.546Z,NaN,NaN,NaN,5
53447,Co.opmart Thủ Dầu Một,2025-05-20T10:54:18.780Z,NaN,NaN,NaN,5
53448,Co.opmart Thủ Dầu Một,2025-05-14T13:11:58.722Z,NaN,NaN,NaN,5
53449,Co.opmart Thủ Dầu Một,2025-05-11T02:18:15.470Z,NaN,NaN,NaN,5
53450,Co.opmart Thủ Dầu Một,2025-05-08T07:48:29.705Z,NaN,NaN,NaN,5


## CHUẨN HÓA TẠO CỘT COMMENT

In [7]:
df['comment'] = (df['text'].where(df['originalLanguage'] == 'vi', df['textTranslated']))

In [8]:
df['comment'] = (df['comment'].astype(str))

# CHUẨN HÓA DỮ LIỆU COMMENT

In [9]:
ACRONYMS = {
    "r": "rồi", "a": "anh", "e": "em", "j": "gì", "k": "không", "m": "mình",
    "t": "tôi", "b": "bạn", "h": "giờ", "s": "sao",

    "ko": "không", "k0": "không", "bt": "bình thường", "vn": "việt nam", "vs": "và",
    "cx": "cũng được", "đc": "được", "dc": "được", "nh": "nhưng", "đg": "đường",
    "nc": "nước", "ms": "mới", "bh": "bao giờ", "km": "khuyến mãi", "ae": "anh em",
    "sg": "sài gòn", "hn": "hà nội", "vk": "vợ", "ck": "chồng", "nv": "nhân viên",
    "mn": "mọi người", "qc": "quảng cáo", "sp": "sản phẩm", "sd": "sử dụng", "wa": "quá",

    "ntn": "như thế nào", "lun": "luôn", "trc": "trước", "chx": "chưa", "nhg": "nhưng",
    "spt": "sản phẩm tốt", "nch": "nói chung", "kbh": "không bao giờ",

    "tnao": "thế nào", "hqua": "hôm qua", "toẹt": "tuyệt", "nhưg": "nhưng",
    "hnay": "hôm nay", "thik": "thích",

    "lquan": "liên quan", "nchung": "nói chung", "k_thể": "không thể"
}

def replace_acronyms(text):
    for abbr, full in ACRONYMS.items():
        text = re.sub(rf'\b{abbr}\b', full, text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile(
        "[" +
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002700-\U000027BF"
        "\U0001F900-\U0001F9FF"
        "\U0001FA70-\U0001FAFF"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub('', text)

def remove_special_characters(text):
    text = re.sub(r"http\S+|www\S+|@\S+|#\S+", " ", text)
    text = re.sub(r"[^a-zA-ZÀ-ỹà-ỹ\s]", " ", text)
    return text

def normalize_vietnamese(text):
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ',
        'òe': 'oè', 'óe': 'oé', 'ỏe': 'oẻ', 'õe': 'oẽ', 'ọe': 'oẹ',
        'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ', 'ụy': 'uỵ',
        'uả': 'ủa', 'ả': 'ả', 'ố': 'ố', 'u´': 'ố', 'ỗ': 'ỗ', 'ồ': 'ồ',
        'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ', 'ầ': 'ầ', 'ỏ': 'ỏ',
        'ề': 'ề', 'ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở',
        'ỉ': 'ỉ', 'ẻ': 'ẻ', 'àk': ' à ', 'aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ',
        'ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á'
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)
    text = re.sub(r'([a-zA-Zà-ỹ])\1{2,}', r'\1', text)
    return text

def preprocess_text(text):
    if text is None or (isinstance(text, float) and pd.isna(text)):
        return ""

    text = text.lower()
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    text = remove_emoji(text)
    text = normalize_vietnamese(text)
    text = remove_special_characters(text)
    text = replace_acronyms(text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = ViTokenizer.tokenize(text)
    return text


In [10]:
df['comment'] = df['comment'].apply(preprocess_text)


In [11]:
df['comment'] = df['comment'].fillna('').astype(str)
df['comment'] = df['comment'].apply(preprocess_text)
df['comment'] = df['comment'].replace('nan', '')
df = df[df['comment'].str.strip() != '']
df = df[df['comment'].str.split().str.len() >= 3]
df

,title,publishedAtDate,originalLanguage,textTranslated,text,stars,comment
0,Siêu thị Co.opmart Nha Trang,2025-06-14T08:20:44.449Z,vi,NaN,"Khu vui chơi nóng mặc dù rất ít trẻ nhỏ chơi, ...",1,khu vui_chơi nóng mặc_dù rất ít trẻ nhỏ chơi c...
1,Siêu thị Co.opmart Nha Trang,2025-09-05T01:15:13.880Z,vi,NaN,cái quầy giữ đồ của khách làm ăn khó chịu thật...,1,cái quầy giữ đồ của khách làm_ăn khó_chịu thật...
2,Siêu thị Co.opmart Nha Trang,2025-10-18T02:34:51.862Z,vi,NaN,Mua hàng online. Mặt hàng để còn hàng nhưng lú...,1,mua hàng online mặt_hàng để còn hàng nhưng lúc...
3,Siêu thị Co.opmart Nha Trang,2025-06-21T12:36:28.800Z,vi,NaN,"Bán hàng thì đắt hon các siêu thị khác, giữ xe...",1,bán hàng thì đắt hon các siêu_thị khác giữ xe ...
4,Siêu thị Co.opmart Nha Trang,2025-07-07T12:21:30.734Z,vi,NaN,- Siêu thị duy nhất thu tiền giữ xe\n- Nhân vi...,1,siêu_thị duy_nhất thu tiền giữ xe nhân_viên câ...
...,...,...,...,...,...,...,...
53370,Co.opmart Thủ Dầu Một,2018-10-02T08:54:16.001Z,en,chợ thực phẩm sạch,clean grocery market,5,chợ thực_phẩm sạch
53372,Co.opmart Thủ Dầu Một,2017-12-30T09:21:02.347Z,en,Siêu thị hàng ngày của tôi,My everyday supermarket,5,siêu_thị hàng ngày của tôi
53373,Co.opmart Thủ Dầu Một,2016-09-26T15:04:17.051Z,en,Co.opmart rất tốt!!!,Co.opmart very good!!!,5,co opmart rất tốt
53426,Co.opmart Bà Rịa,2019-08-25T08:48:59.797Z,en,Thích hợp cho việc mua sắm,Good for shopping,5,thích_hợp cho việc mua_sắm


In [22]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 1. Chuẩn bị dữ liệu: Gộp tất cả văn bản thành một chuỗi lớn
# Bạn có thể lọc theo label (ví dụ: chỉ vẽ cho các bình luận tích cực)
text_data = " ".join(review for review in df.comment.astype(str))

# 2. Định nghĩa danh sách Stopwords (Từ vô nghĩa) tiếng Việt
# Bạn nên bổ sung thêm các từ xuất hiện nhiều nhưng không mang ý nghĩa ở đây
vietnamese_stopwords = ["và", "là", "của", "cho", "có", "rất", "này", "được", "mình", "shop"]

# 3. Khởi tạo WordCloud
wordcloud = WordCloud(
    font_path=None, # Nếu bị lỗi font tiếng Việt, bạn hãy trỏ đường dẫn đến file .ttf (VD: 'Arial.ttf')
    width=700, 
    height=300, 
    background_color='white',
    stopwords=vietnamese_stopwords,
    max_words=200,
    colormap='viridis' # Bạn có thể đổi thành 'coolwarm', 'magma', 'Plasma'
).generate(text_data)

# 4. Hiển thị
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off") # Ẩn trục tọa độ
plt.show()

# GẮN NHÃN DỮ LIỆU

In [18]:
df2 = df[['comment',"stars"]]
df2

,comment,stars
0,khu vui_chơi nóng mặc_dù rất ít trẻ nhỏ chơi c...,1
1,cái quầy giữ đồ của khách làm_ăn khó_chịu thật...,1
2,mua hàng online mặt_hàng để còn hàng nhưng lúc...,1
3,bán hàng thì đắt hon các siêu_thị khác giữ xe ...,1
4,siêu_thị duy_nhất thu tiền giữ xe nhân_viên câ...,1
...,...,...
53370,chợ thực_phẩm sạch,5
53372,siêu_thị hàng ngày của tôi,5
53373,co opmart rất tốt,5
53426,thích_hợp cho việc mua_sắm,5


In [19]:
checkpoint = "5CD-AI/Vietnamese-Sentiment-visobert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

def get_sentiments_in_batches(texts, batch_size=128):
    sentiment_mapping = {"POS": "Tích cực", "NEU": "Trung tính", "NEG": "Tiêu cực"}
    model.eval()
    all_sentiments = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

            for pred in predictions:
                label = model.config.id2label[pred.item()]
                sentiment = sentiment_mapping.get(label, "Không xác định")
                all_sentiments.append(sentiment)

    return all_sentiments




In [21]:
df2['sentiment'] = get_sentiments_in_batches(df2['comment'].tolist(), batch_size=128)

100%|██████████| 149/149 [58:06<00:00, 23.40s/it]   
C:\Users\USER\AppData\Local\Temp\ipykernel_17420\1027846204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['sentiment'] = get_sentiments_in_batches(df2['comment'].tolist(), batch_size=128)


In [23]:
df2

,comment,stars,sentiment
0,khu vui_chơi nóng mặc_dù rất ít trẻ nhỏ chơi c...,1,Tiêu cực
1,cái quầy giữ đồ của khách làm_ăn khó_chịu thật...,1,Tiêu cực
2,mua hàng online mặt_hàng để còn hàng nhưng lúc...,1,Tiêu cực
3,bán hàng thì đắt hon các siêu_thị khác giữ xe ...,1,Tiêu cực
4,siêu_thị duy_nhất thu tiền giữ xe nhân_viên câ...,1,Tiêu cực
...,...,...,...
53370,chợ thực_phẩm sạch,5,Tích cực
53372,siêu_thị hàng ngày của tôi,5,Tích cực
53373,co opmart rất tốt,5,Tích cực
53426,thích_hợp cho việc mua_sắm,5,Tích cực


Đang trích xuất tính từ, vui lòng đợi...


In [36]:
from pyvi import ViPosTagger, ViTokenizer

def extract_adjectives(text):
    # Tách từ và gán nhãn loại từ
    tokens = ViPosTagger.postagging(ViTokenizer.tokenize(str(text)))
    # tokens[0] là danh sách từ, tokens[1] là danh sách nhãn (A, V, N,...)
    adjectives = [word for word, tag in zip(tokens[0], tokens[1]) if tag == 'A']
    return " ".join(adjectives)

# 1. Lọc lấy tính từ cho từng nhóm (có thể mất một chút thời gian nếu dữ liệu lớn)
print("Đang trích xuất tính từ, vui lòng đợi...")
pos_adj_text = " ".join(df2[df2['sentiment'] == 'Tích cực']['comment'].apply(extract_adjectives))
neg_adj_text = " ".join(df2[df2['sentiment'] == 'Tiêu cực']['comment'].apply(extract_adjectives))

# 2. Vẽ Word Cloud từ danh sách tính từ này
wc_params = {'width': 800, 'height': 800, 'background_color': 'white', 'max_words': 50}

wordcloud_pos = WordCloud(**wc_params, colormap='Greens').generate(pos_adj_text)
wordcloud_neg = WordCloud(**wc_params, colormap='Reds').generate(neg_adj_text)

# 3. Hiển thị
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_pos)
plt.title("Tích cực", fontsize=25, color='green')
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_neg)
plt.title("Tiêu cực", fontsize=25, color='red')
plt.axis("off")
plt.show()

C:\Users\USER\AppData\Local\Temp\ipykernel_11376\344306660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:, "length"] = df2["comment"].str.len()


sentiment
Tiêu cực      143.142334
Trung tính     46.642223
Tích cực       70.400119
Name: length, dtype: float64

In [16]:
df2.loc[:, "length"] = df2["comment"].str.len()

plt.figure(figsize=(6,4))
sns.countplot(x="sentiment", data=df2)
plt.title("Phân bố nhãn Sentiment")
plt.show()

df2.groupby("sentiment")["length"].mean()

# VECTOR HÓA

In [17]:
vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,2))

X = vectorizer.fit_transform(df2["comment"])
y = df2["sentiment"]

# MACHINE LEARNING

## CHIA TỆP DỮ LIỆU

In [18]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## HUẤN LUYỆN MÔ HÌNH HỌC MÁY

In [19]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [20]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [21]:
svm = LinearSVC()
svm.fit(X_train, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None


In [22]:
models = {
    "Logistic Regression": lr,
    "Naive Bayes": nb,
    "SVM": svm
}

for name, model_ in models.items():
    print(f"\n{name}")
    y_pred = model_.predict(X_test)
    print(classification_report(y_test, y_pred))


Logistic Regression
              precision    recall  f1-score   support

    Tiêu cực       0.82      0.83      0.83      1365
  Trung tính       0.63      0.26      0.37       428
    Tích cực       0.83      0.93      0.88      2008

    accuracy                           0.82      3801
   macro avg       0.76      0.67      0.69      3801
weighted avg       0.81      0.82      0.80      3801


Naive Bayes
              precision    recall  f1-score   support

    Tiêu cực       0.79      0.78      0.78      1365
  Trung tính       0.66      0.11      0.18       428
    Tích cực       0.78      0.92      0.84      2008

    accuracy                           0.78      3801
   macro avg       0.74      0.60      0.60      3801
weighted avg       0.77      0.78      0.75      3801


SVM
              precision    recall  f1-score   support

    Tiêu cực       0.83      0.83      0.83      1365
  Trung tính       0.58      0.36      0.44       428
    Tích cực       0.85      0.91   

# DEEP LEARNING

## CHIA TỆP DỮ LIỆU

In [23]:
X_dense = X.toarray()

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_dense, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [24]:
model_dl = Sequential([Dense(256, activation="relu", input_shape=(X_train_d.shape[1],)),Dropout(0.3),Dense(128, activation="relu"),Dense(len(le.classes_), activation="softmax")])

model_dl.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

model_dl.summary()

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     1,280,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,539 (5.01 MB)

 Trainable params: 1,313,539 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model_dl.fit(X_train_d,y_train_d,epochs=10,batch_size=32,validation_split=0.1)

Epoch 1/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7742 - loss: 0.5645 - val_accuracy: 0.8014 - val_loss: 0.4943
Epoch 2/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8653 - loss: 0.3453 - val_accuracy: 0.8087 - val_loss: 0.5025
Epoch 3/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9154 - loss: 0.2303 - val_accuracy: 0.8008 - val_loss: 0.6009
Epoch 4/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9525 - loss: 0.1403 - val_accuracy: 0.7909 - val_loss: 0.7128
Epoch 5/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9733 - loss: 0.0796 - val_accuracy: 0.7909 - val_loss: 0.8750
Epoch 6/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9852 - loss: 0.0470 - val_accuracy: 0.7988 - val_loss: 0.9936
Epoch 7/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9896 - loss: 0.0316 - val_accuracy: 0.8041 - val_loss: 1.1432
Epoch 8/10
428/428 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9928 - loss: 0.0233 - val_accuracy: 0.

In [26]:
y_pred_dl = model_dl.predict(X_test_d)
y_pred_dl = np.argmax(y_pred_dl, axis=1)

print(classification_report(y_test_d, y_pred_dl, target_names=le.classes_))

119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

    Tiêu cực       0.79      0.83      0.81      1365
  Trung tính       0.48      0.37      0.42       428
    Tích cực       0.85      0.87      0.86      2008

    accuracy                           0.80      3801
   macro avg       0.71      0.69      0.70      3801
weighted avg       0.79      0.80      0.79      3801



In [27]:
def predict_sentiment_app(text):
    text = preprocess_text(text)
    vec = vectorizer.transform([text])
    return svm.predict(vec)[0]

predict_sentiment_app("Sản phẩm rất tốt, giao hàng nhanh")


'Tích cực'

In [35]:
import joblib
# Kế thừa đúng tên biến từ file của bạn
joblib.dump(svm, 'svm_model.pkl') 
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Đã xuất model và vectorizer thành công!")

Đã xuất model và vectorizer thành công!
